In [20]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import torch
import numpy as np
from glob import glob
from sklearn.metrics.pairwise import cosine_similarity


In [8]:
wadi_sensors = [
    "Row", "Date", "Time", "1_AIT_001_PV", "1_AIT_002_PV", "1_AIT_003_PV",
    "1_AIT_004_PV", "1_AIT_005_PV", "1_FIT_001_PV", "1_LS_001_AL", "1_LS_002_AL",
    "1_LT_001_PV", "1_MV_001_STATUS", "1_MV_002_STATUS", "1_MV_003_STATUS",
    "1_MV_004_STATUS", "1_P_001_STATUS", "1_P_002_STATUS", "1_P_003_STATUS",
    "1_P_004_STATUS", "1_P_005_STATUS", "1_P_006_STATUS", "2_DPIT_001_PV",
    "2_FIC_101_CO", "2_FIC_101_PV", "2_FIC_101_SP", "2_FIC_201_CO", "2_FIC_201_PV",
    "2_FIC_201_SP", "2_FIC_301_CO", "2_FIC_301_PV", "2_FIC_301_SP", "2_FIC_401_CO",
    "2_FIC_401_PV", "2_FIC_401_SP", "2_FIC_501_CO", "2_FIC_501_PV", "2_FIC_501_SP",
    "2_FIC_601_CO", "2_FIC_601_PV", "2_FIC_601_SP", "2_FIT_001_PV", "2_FIT_002_PV",
    "2_FIT_003_PV", "2_FQ_101_PV", "2_FQ_201_PV", "2_FQ_301_PV", "2_FQ_401_PV",
    "2_FQ_501_PV", "2_FQ_601_PV", "2_LS_001_AL", "2_LS_002_AL", "2_LS_101_AH",
    "2_LS_101_AL", "2_LS_201_AH", "2_LS_201_AL", "2_LS_301_AH", "2_LS_301_AL",
    "2_LS_401_AH", "2_LS_401_AL", "2_LS_501_AH", "2_LS_501_AL", "2_LS_601_AH",
    "2_LS_601_AL", "2_LT_001_PV", "2_LT_002_PV", "2_MCV_007_CO", "2_MCV_101_CO",
    "2_MCV_201_CO", "2_MCV_301_CO", "2_MCV_401_CO", "2_MCV_501_CO", "2_MCV_601_CO",
    "2_MV_001_STATUS", "2_MV_002_STATUS", "2_MV_003_STATUS", "2_MV_004_STATUS",
    "2_MV_005_STATUS", "2_MV_006_STATUS", "2_MV_009_STATUS", "2_MV_101_STATUS",
    "2_MV_201_STATUS", "2_MV_301_STATUS", "2_MV_401_STATUS", "2_MV_501_STATUS",
    "2_MV_601_STATUS", "2_P_001_STATUS", "2_P_002_STATUS", "2_P_003_SPEED",
    "2_P_003_STATUS", "2_P_004_SPEED", "2_P_004_STATUS", "2_PIC_003_CO",
    "2_PIC_003_PV", "2_PIC_003_SP", "2_PIT_001_PV", "2_PIT_002_PV", "2_PIT_003_PV",
    "2_SV_101_STATUS", "2_SV_201_STATUS", "2_SV_301_STATUS", "2_SV_401_STATUS",
    "2_SV_501_STATUS", "2_SV_601_STATUS", "2A_AIT_001_PV", "2A_AIT_002_PV",
    "2A_AIT_003_PV", "2A_AIT_004_PV", "2B_AIT_001_PV", "2B_AIT_002_PV",
    "2B_AIT_003_PV", "2B_AIT_004_PV", "3_AIT_001_PV", "3_AIT_002_PV",
    "3_AIT_003_PV", "3_AIT_004_PV", "3_AIT_005_PV", "3_FIT_001_PV", "3_LS_001_AL",
    "3_LT_001_PV", "3_MV_001_STATUS", "3_MV_002_STATUS", "3_MV_003_STATUS",
    "3_P_001_STATUS", "3_P_002_STATUS", "3_P_003_STATUS", "3_P_004_STATUS",
    "LEAK_DIFF_PRESSURE", "PLANT_START_STOP_LOG", "TOTAL_CONS_REQUIRED_FLOW"
]
swat_sensors = [
    "Timestamp", "FIT101", "LIT101", "MV101", "P101", "P102", "AIT201", "AIT202",
    "AIT203", "FIT201", "MV201", "P201", "P202", "P203", "P204", "P205", "P206",
    "DPIT301", "FIT301", "LIT301", "MV301", "MV302", "MV303", "MV304", "P301",
    "P302", "AIT401", "AIT402", "FIT401", "LIT401", "P401", "P402", "P403", "P404",
    "UV401", "AIT501", "AIT502", "AIT503", "AIT504", "FIT501", "FIT502", "FIT503",
    "FIT504", "P501", "P502", "PIT501", "PIT502", "PIT503", "FIT601", "P601",
    "P602", "P603", "Normal/Attack"
]

# Create a dictionary with index as key and [sensor_name, type, group] as value
swat_sensors_dict = {
    0: ["FIT101", "Sensor", 1],
    1: ["LIT101", "Sensor", 1],
    2: ["MV101", "Actuator", 1],
    3: ["P101", "Actuator", 1],
    4: ["P102", "Actuator", 1],
    5: ["AIT201", "Sensor", 2],
    6: ["AIT202", "Sensor", 2],
    7: ["AIT203", "Sensor", 2],
    8: ["FIT201", "Sensor", 2],
    9: ["MV201", "Actuator", 2],
    10: ["P201", "Actuator", 2],
    11: ["P202", "Actuator", 2],
    12: ["P203", "Actuator", 2],
    13: ["P204", "Actuator", 2],
    14: ["P205", "Actuator", 2],
    15: ["P206", "Actuator", 2],
    16: ["DPIT301", "Sensor", 3],
    17: ["FIT301", "Sensor", 3],
    18: ["LIT301", "Sensor", 3],
    19: ["MV301", "Actuator", 3],
    20: ["MV302", "Actuator", 3],
    21: ["MV303", "Actuator", 3],
    22: ["MV304", "Actuator", 3],
    23: ["P301", "Actuator", 3],
    24: ["P302", "Actuator", 3],
    25: ["AIT401", "Sensor", 4],
    26: ["AIT402", "Sensor", 4],
    27: ["FIT401", "Sensor", 4],
    28: ["LIT401", "Actuator", 4],  # Note: LIT401 is listed as an Actuator in your data, though LIT typically indicates a sensor
    29: ["P401", "Actuator", 4],
    30: ["P402", "Actuator", 4],
    31: ["P403", "Actuator", 4],
    32: ["P404", "Actuator", 4],
    33: ["UV401", "Actuator", 4],
    34: ["AIT501", "Sensor", 5],
    35: ["AIT502", "Sensor", 5],
    36: ["AIT503", "Sensor", 5],
    37: ["AIT504", "Sensor", 5],
    38: ["FIT501", "Sensor", 5],
    39: ["FIT502", "Sensor", 5],
    40: ["FIT503", "Sensor", 5],
    41: ["FIT504", "Sensor", 5],
    42: ["P501", "Actuator", 5],
    43: ["P502", "Actuator", 5],
    44: ["PIT501", "Sensor", 5],
    45: ["PIT502", "Sensor", 5],
    46: ["PIT503", "Sensor", 5],
    47: ["FIT601", "Sensor", 6],
    48: ["P601", "Actuator", 6],
    49: ["P602", "Actuator", 6],
    50: ["P603", "Actuator", 6]
}

wadi_dict = {
    # Primary Grid (P1)
    0: ["1_AIT_001_PV", "Sensor", "P1"],  # Water quality sensor
    1: ["1_AIT_002_PV", "Sensor", "P1"],  # Water quality sensor
    2: ["1_AIT_003_PV", "Sensor", "P1"],  # Water quality sensor
    3: ["1_AIT_004_PV", "Sensor", "P1"],  # Water quality sensor
    4: ["1_AIT_005_PV", "Sensor", "P1"],  # Water quality sensor
    5: ["1_FIT_001_PV", "Sensor", "P1"],  # Flow meter
    6: ["1_LS_001_AL", "Sensor", "P1"],   # Level switch alarm
    7: ["1_LS_002_AL", "Sensor", "P1"],   # Level switch alarm
    8: ["1_LT_001_PV", "Sensor", "P1"],   # Level transmitter
    9: ["1_MV_001_STATUS", "Actuator", "P1"],  # Motorized valve
    10: ["1_MV_002_STATUS", "Actuator", "P1"],  # Motorized valve
    11: ["1_MV_003_STATUS", "Actuator", "P1"],  # Motorized valve
    12: ["1_MV_004_STATUS", "Actuator", "P1"],  # Motorized valve
    13: ["1_P_001_STATUS", "Actuator", "P1"],   # Pump
    14: ["1_P_002_STATUS", "Actuator", "P1"],   # Pump
    15: ["1_P_003_STATUS", "Actuator", "P1"],   # Pump
    16: ["1_P_004_STATUS", "Actuator", "P1"],   # Pump
    17: ["1_P_005_STATUS", "Actuator", "P1"],   # Pump
    18: ["1_P_006_STATUS", "Actuator", "P1"],   # Pump

    # Secondary Grid Part A (P2A - Elevated Reservoir)
    19: ["2_DPIT_001_PV", "Sensor", "P2A"],  # Differential pressure indicator
    20: ["2_FIT_001_PV", "Sensor", "P2A"],   # Flow meter
    21: ["2_FIT_002_PV", "Sensor", "P2A"],   # Flow meter
    22: ["2_FIT_003_PV", "Sensor", "P2A"],   # Flow meter
    23: ["2_LT_001_PV", "Sensor", "P2A"],    # Level transmitter - elevated reservoir
    24: ["2_LT_002_PV", "Sensor", "P2A"],    # Level transmitter - elevated reservoir
    25: ["2_MV_001_STATUS", "Actuator", "P2A"],  # Motorized valve
    26: ["2_MV_002_STATUS", "Actuator", "P2A"],  # Motorized valve
    27: ["2_MV_003_STATUS", "Actuator", "P2A"],  # Motorized valve
    28: ["2_MV_004_STATUS", "Actuator", "P2A"],  # Motorized valve
    29: ["2_MV_005_STATUS", "Actuator", "P2A"],  # Motorized valve
    30: ["2_MV_006_STATUS", "Actuator", "P2A"],  # Motorized valve
    31: ["2_MV_009_STATUS", "Actuator", "P2A"],  # Motorized valve
    32: ["2_P_001_STATUS", "Actuator", "P2A"],   # Pump
    33: ["2_P_002_STATUS", "Actuator", "P2A"],   # Pump
    34: ["2_PIT_001_PV", "Sensor", "P2A"],       # Pressure indicator
    35: ["2_PIT_002_PV", "Sensor", "P2A"],       # Pressure indicator
    36: ["2_PIT_003_PV", "Sensor", "P2A"],       # Pressure indicator
    37: ["2A_AIT_001_PV", "Sensor", "P2A"],      # Water quality sensor
    38: ["2A_AIT_002_PV", "Sensor", "P2A"],      # Water quality sensor
    39: ["2A_AIT_003_PV", "Sensor", "P2A"],      # Water quality sensor
    40: ["2A_AIT_004_PV", "Sensor", "P2A"],      # Water quality sensor

    # Secondary Grid Part B (P2B - Consumer Tanks)
    41: ["2_FIC_101_PV", "Sensor", "P2B"],       # Flow indicator/controller
    42: ["2_FIC_201_PV", "Sensor", "P2B"],       # Flow indicator/controller
    43: ["2_FIC_301_PV", "Sensor", "P2B"],       # Flow indicator/controller
    44: ["2_FIC_401_PV", "Sensor", "P2B"],       # Flow indicator/controller
    45: ["2_FIC_501_PV", "Sensor", "P2B"],       # Flow indicator/controller
    46: ["2_FIC_601_PV", "Sensor", "P2B"],       # Flow indicator/controller
    47: ["2_FQ_101_PV", "Sensor", "P2B"],        # Flow totalizer
    48: ["2_FQ_201_PV", "Sensor", "P2B"],        # Flow totalizer
    49: ["2_FQ_301_PV", "Sensor", "P2B"],        # Flow totalizer
    50: ["2_FQ_401_PV", "Sensor", "P2B"],        # Flow totalizer
    51: ["2_FQ_501_PV", "Sensor", "P2B"],        # Flow totalizer
    52: ["2_FQ_601_PV", "Sensor", "P2B"],        # Flow totalizer
    53: ["2_LS_101_AH", "Sensor", "P2B"],        # Level switch high alarm
    54: ["2_LS_101_AL", "Sensor", "P2B"],        # Level switch low alarm
    55: ["2_LS_201_AH", "Sensor", "P2B"],        # Level switch high alarm
    56: ["2_LS_201_AL", "Sensor", "P2B"],        # Level switch low alarm
    57: ["2_LS_301_AH", "Sensor", "P2B"],        # Level switch high alarm
    58: ["2_LS_301_AL", "Sensor", "P2B"],        # Level switch low alarm
    59: ["2_LS_401_AH", "Sensor", "P2B"],        # Level switch high alarm
    60: ["2_LS_401_AL", "Sensor", "P2B"],        # Level switch low alarm
    61: ["2_LS_501_AH", "Sensor", "P2B"],        # Level switch high alarm
    62: ["2_LS_501_AL", "Sensor", "P2B"],        # Level switch low alarm
    63: ["2_LS_601_AH", "Sensor", "P2B"],        # Level switch high alarm
    64: ["2_LS_601_AL", "Sensor", "P2B"],        # Level switch low alarm
    65: ["2_MCV_101_STATUS", "Actuator", "P2B"], # Motorized consumer valve
    66: ["2_MCV_201_STATUS", "Actuator", "P2B"], # Motorized consumer valve
    67: ["2_MCV_301_STATUS", "Actuator", "P2B"], # Motorized consumer valve
    68: ["2_MCV_401_STATUS", "Actuator", "P2B"], # Motorized consumer valve
    69: ["2_MCV_501_STATUS", "Actuator", "P2B"], # Motorized consumer valve
    70: ["2_MCV_601_STATUS", "Actuator", "P2B"], # Motorized consumer valve
    71: ["2_SV_101_STATUS", "Actuator", "P2B"],  # Solenoid valve
    72: ["2_SV_201_STATUS", "Actuator", "P2B"],  # Solenoid valve
    73: ["2_SV_301_STATUS", "Actuator", "P2B"],  # Solenoid valve
    74: ["2_SV_401_STATUS", "Actuator", "P2B"],  # Solenoid valve
    75: ["2_SV_501_STATUS", "Actuator", "P2B"],  # Solenoid valve
    76: ["2_SV_601_STATUS", "Actuator", "P2B"],  # Solenoid valve
    77: ["2B_AIT_001_PV", "Sensor", "P2B"],      # Water quality sensor
    78: ["2B_AIT_002_PV", "Sensor", "P2B"],      # Water quality sensor
    79: ["2B_AIT_003_PV", "Sensor", "P2B"],      # Water quality sensor
    80: ["2B_AIT_004_PV", "Sensor", "P2B"],      # Water quality sensor

    # Return Water Grid (P3)
    81: ["3_AIT_001_PV", "Sensor", "P3"],        # Water quality sensor
    82: ["3_AIT_002_PV", "Sensor", "P3"],        # Water quality sensor
    83: ["3_AIT_003_PV", "Sensor", "P3"],        # Water quality sensor
    84: ["3_AIT_004_PV", "Sensor", "P3"],        # Water quality sensor
    85: ["3_AIT_005_PV", "Sensor", "P3"],        # Water quality sensor
    86: ["3_FIT_001_PV", "Sensor", "P3"],        # Flow meter
    87: ["3_LS_001_AL", "Sensor", "P3"],         # Level switch alarm
    88: ["3_LT_001_PV", "Sensor", "P3"],         # Level transmitter
    89: ["3_MV_001_STATUS", "Actuator", "P3"],   # Motorized valve
    90: ["3_MV_002_STATUS", "Actuator", "P3"],   # Motorized valve
    91: ["3_MV_003_STATUS", "Actuator", "P3"],   # Motorized valve
    92: ["3_P_001_STATUS", "Actuator", "P3"],    # Pump
    93: ["3_P_002_STATUS", "Actuator", "P3"],    # Pump
    94: ["3_P_003_STATUS", "Actuator", "P3"],    # Pump
    95: ["3_P_004_STATUS", "Actuator", "P3"],    # Pump
}
# Example usage: access information about a sensor
# print(swat_sensors_dict[38])  # Output: ['FIT501', 'Sensor', 5]

### Data sanity check

In [9]:
# Load the .npy file
data = np.load('/home/berentzenaej/baselines/GTA/results/gta_SWaT_ftM_sl60_ll30_pl30_nl3_dm128_nh8_el3_dl2_df128_atprob_ebfixed_swat_test_0/embeddings/adjacency_matrix.npy')

# Print array information
print("Array shape:", data.shape)
print("Array dtype:", data.dtype)

# For small arrays, print the entire array
if data.size < 1000:  # Adjust this threshold as needed
    print("Array contents:")
    print(data)
else:
    # For larger arrays, print summary statistics and samples
    print("Array summary:")
    print("Min:", data.min())
    print("Max:", data.max())
    print("Mean:", data.mean())
    print("First few elements:", data.flatten()[:10])
    print("Last few elements:", data.flatten()[-10:])

    # If the array is multi-dimensional, print a few slices
    if data.ndim > 2:
        print("\nSample slices:")
        for i in range(min(3, data.shape[0])):
            print(f"Slice {i}:")
            print(data[i])

Array shape: (51, 51)
Array dtype: float64
Array summary:
Min: 0.6518000053174405
Max: 0.6518000185702205
Mean: 0.6518000128284799
First few elements: [0.65180001 0.65180001 0.65180001 0.65180001 0.65180001 0.65180001
 0.65180001 0.65180001 0.65180001 0.65180001]
Last few elements: [0.65180001 0.65180002 0.65180002 0.65180001 0.65180001 0.65180001
 0.65180001 0.65180001 0.65180001 0.65180001]


In [14]:
def build_tensor():
    # Expand the tilde to the home directory
    pattern = os.path.expanduser(f"~/baselines/GTA/results/gta_SWaT_ftM_sl60_ll30_pl30_nl3_dm128_nh8_el3_dl2_df128_atprob_ebfixed_swat_test_*/embeddings/adjacency_matrix.npy")

    # Find all matching files
    file_paths = sorted(glob(pattern))
    print(f"Found {len(file_paths)} files matching the pattern")

    # Function to load a matrix from a text file
    def load_matrix(file_path):
        return np.load(file_path)


    # Load all matrices into a tensor
    matrices = []
    for file_path in file_paths:
        print(f"Loading {file_path}...")
        matrix = load_matrix(file_path)
        matrices.append(matrix)
        print(f"Loaded matrix with shape {matrix.shape}")

    # Stack all matrices into a single tensor
    if matrices:
        learned_graph_tensor = np.stack(matrices)
        print(f"Created tensor with shape {learned_graph_tensor.shape}")
    else:
        print("No matrices found")
    return learned_graph_tensor
swat_tensor = build_tensor()

Found 2 files matching the pattern
Loading /home/berentzenaej/baselines/GTA/results/gta_SWaT_ftM_sl60_ll30_pl30_nl3_dm128_nh8_el3_dl2_df128_atprob_ebfixed_swat_test_0/embeddings/adjacency_matrix.npy...
Loaded matrix with shape (51, 51)
Loading /home/berentzenaej/baselines/GTA/results/gta_SWaT_ftM_sl60_ll30_pl30_nl3_dm128_nh8_el3_dl2_df128_atprob_ebfixed_swat_test_1/embeddings/adjacency_matrix.npy...
Loaded matrix with shape (51, 51)
Created tensor with shape (2, 51, 51)


### Structural consistency

In [21]:
def calculate_cosine_similarities(matrices):
    # Calculate cosine similarities between all pairs of matrices
    
    num_matrices = len(matrices)
    all_similarities = []

    for i in range(num_matrices):
        for j in range(i+1, num_matrices):
            # Flatten matrices for cosine similarity calculation
            matrix_i_flat = matrices[i].reshape(1, -1)
            matrix_j_flat = matrices[j].reshape(1, -1)

            sim = cosine_similarity(matrix_i_flat, matrix_j_flat)[0, 0]
            all_similarities.append(sim)
            print(f"Cosine similarity between matrix {i} and {j}: {sim:.4f}")

    # Calculate average similarity
    avg_similarity = np.mean(all_similarities) if all_similarities else None
    print(f"Average cosine similarity: {avg_similarity:.4f}")

    # Calculate cosine similarity for each sensor (row in the matrix)
    sensor_similarities = []

    # Check if matrices have the same shape
    if all(m.shape == matrices[0].shape for m in matrices):
        num_sensors = matrices[0].shape[0]
        for sensor_idx in range(num_sensors):
            sensor_sims = []
            for i in range(num_matrices):
                for j in range(i+1, num_matrices):
                    # Get the row for this sensor from both matrices
                    sensor_i = matrices[i][sensor_idx, :].reshape(1, -1)
                    sensor_j = matrices[j][sensor_idx, :].reshape(1, -1)

                    sim = cosine_similarity(sensor_i, sensor_j)[0, 0]
                    sensor_sims.append(sim)

            # Average similarity for this sensor
            avg_sensor_sim = np.mean(sensor_sims) if sensor_sims else None
            sensor_similarities.append(avg_sensor_sim)
            print(f"Sensor {swat_sensors_dict[sensor_idx][0]} average similarity: {avg_sensor_sim:.4f}")
    else:
        print("Cannot calculate per-sensor similarities: matrices have different shapes")

    return all_similarities, avg_similarity, sensor_similarities

# Use the pattern provided
all_sims, avg_sim, sensor_sims = calculate_cosine_similarities(swat_tensor)

Cosine similarity between matrix 0 and 1: 1.0000
Average cosine similarity: 1.0000
Sensor FIT101 average similarity: 1.0000
Sensor LIT101 average similarity: 1.0000
Sensor MV101 average similarity: 1.0000
Sensor P101 average similarity: 1.0000
Sensor P102 average similarity: 1.0000
Sensor AIT201 average similarity: 1.0000
Sensor AIT202 average similarity: 1.0000
Sensor AIT203 average similarity: 1.0000
Sensor FIT201 average similarity: 1.0000
Sensor MV201 average similarity: 1.0000
Sensor P201 average similarity: 1.0000
Sensor P202 average similarity: 1.0000
Sensor P203 average similarity: 1.0000
Sensor P204 average similarity: 1.0000
Sensor P205 average similarity: 1.0000
Sensor P206 average similarity: 1.0000
Sensor DPIT301 average similarity: 1.0000
Sensor FIT301 average similarity: 1.0000
Sensor LIT301 average similarity: 1.0000
Sensor MV301 average similarity: 1.0000
Sensor MV302 average similarity: 1.0000
Sensor MV303 average similarity: 1.0000
Sensor MV304 average similarity: 1.

Node influence distribution

In [18]:
def analyze_node_connectivity(tensor):
    # Load all matching matrices
    results = []

    for matrix in tensor:

        # Calculate degree for each node
        node_degrees = np.sum(matrix, axis=1)

        # Calculate statistics
        degree_mean = np.mean(node_degrees)
        degree_std = np.std(node_degrees)
        degree_variance = np.var(node_degrees)

        # Get highest and lowest connected nodes
        most_connected = np.argmax(node_degrees)
        least_connected = np.argmin(node_degrees)

        results.append({
            'degree_mean': degree_mean,
            'degree_std': degree_std,
            'degree_variance': degree_variance,
            'most_connected': (most_connected, node_degrees[most_connected]),
            'least_connected': (least_connected, node_degrees[least_connected])
        })

        print(f"Mean node degree: {degree_mean:.4f}")
        print(f"Std dev of degrees: {degree_std:.4f}")
        print(f"Variance of degrees: {degree_variance:.4f}")
        print(f"Most connected node: {most_connected} (degree: {node_degrees[most_connected]:.4f})")
        print(f"Least connected node: {least_connected} (degree: {node_degrees[least_connected]:.4f})")

    return results

# Example usage
analysis = analyze_node_connectivity(swat_tensor)

# Calculate overall statistics across all matrices
if analysis:
    means = np.array([result['degree_mean'] for result in analysis])
    stds = np.array([result['degree_std'] for result in analysis])
    variances = np.array([result['degree_variance'] for result in analysis])

    print("\nOverall Statistics:")
    print(f"Mean of means: {np.mean(means):.4f} ± {np.std(means):.4f}")
    print(f"Mean of std devs: {np.mean(stds):.4f} ± {np.std(stds):.4f}")
    print(f"Mean of variances: {np.mean(variances):.4f} ± {np.std(variances):.4f}")

Mean node degree: 33.2418
Std dev of degrees: 0.0000
Variance of degrees: 0.0000
Most connected node: 16 (degree: 33.2418)
Least connected node: 20 (degree: 33.2418)
Mean node degree: 18.5978
Std dev of degrees: 0.0000
Variance of degrees: 0.0000
Most connected node: 30 (degree: 18.5978)
Least connected node: 46 (degree: 18.5978)

Overall Statistics:
Mean of means: 25.9198 ± 7.3220
Mean of std devs: 0.0000 ± 0.0000
Mean of variances: 0.0000 ± 0.0000


### Homophily

In [19]:
import numpy as np
from glob import glob

#TODO test this function!
def calculate_weighted_homophily(tensor, sensor_dictionary, type=1):
    """
    Calculate graph homophily using weighted adjacency matrices.

    Parameters:
    -----------
    pattern : str
        File pattern to match adjacency matrix files.
    sensor_dictionary : dict
        Dictionary mapping node indices to their attributes.
    type : int
        Type of homophily calculation (1 for Type, 2 for Group).

    Returns:
    --------
    list
        List of homophily values for each adjacency matrix.
    """
    # Load all matching matrices
    homophily_values = []

    for adj_matrix in tensor:
        # Ensure the matrix is symmetric (in case it's not)
        adj_matrix = (adj_matrix + adj_matrix.T) / 2

        # Initialize counters
        weighted_total = 0
        weighted_same_attribute = 0

        # Iterate through all node pairs (only upper triangle to avoid double counting)
        for i in range(len(adj_matrix)):
            attr_i = sensor_dictionary[i][type]

            for j in range(i+1, len(adj_matrix)):  # upper triangle only
                attr_j = sensor_dictionary[j][type]

                # Weight of the edge between nodes i and j
                weight = adj_matrix[i, j]

                # Add to total weighted edges
                weighted_total += weight

                # If attributes match, add to same-attribute counter
                if attr_i == attr_j:
                    weighted_same_attribute += weight

        # Calculate homophily for this matrix
        homophily = weighted_same_attribute / weighted_total if weighted_total > 0 else 0
        homophily_values.append(homophily)

        print(f"Weighted homophily (type {type}): {homophily:.4f}")
        print(f"Total weighted edges: {weighted_total:.4f}")
        print(f"Same-attribute weighted edges: {weighted_same_attribute:.4f}")

    return homophily_values

# Calculate homophily for both types
type1_homophily = calculate_weighted_homophily(swat_tensor, swat_sensors_dict, type=1)
type2_homophily = calculate_weighted_homophily(swat_tensor, swat_sensors_dict, type=2)

# Print overall statistics
if type1_homophily:
    print("\nOverall Type 1 Homophily Statistics:")
    print(f"Mean: {np.mean(type1_homophily):.4f} ± {np.std(type1_homophily):.4f}")
    print(f"Min: {np.min(type1_homophily):.4f}, Max: {np.max(type1_homophily):.4f}")

if type2_homophily:
    print("\nOverall Type 2 Homophily Statistics:")
    print(f"Mean: {np.mean(type2_homophily):.4f} ± {np.std(type2_homophily):.4f}")
    print(f"Min: {np.min(type2_homophily):.4f}, Max: {np.max(type2_homophily):.4f}")

Weighted homophily (type 1): 0.4918
Total weighted edges: 831.0450
Same-attribute weighted edges: 408.6786
Weighted homophily (type 1): 0.4918
Total weighted edges: 464.9447
Same-attribute weighted edges: 228.6434
Weighted homophily (type 2): 0.1733
Total weighted edges: 831.0450
Same-attribute weighted edges: 144.0478
Weighted homophily (type 2): 0.1733
Total weighted edges: 464.9447
Same-attribute weighted edges: 80.5904

Overall Type 1 Homophily Statistics:
Mean: 0.4918 ± 0.0000
Min: 0.4918, Max: 0.4918

Overall Type 2 Homophily Statistics:
Mean: 0.1733 ± 0.0000
Min: 0.1733, Max: 0.1733
